In [1]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\bluec\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [3]:
# URL of page to be scraped
# url = 'http://house.speakingsame.com/p.php?q=Perth%2C+WA'
url = 'http://house.speakingsame.com/p.php?q=East+Perth%2C+WA'
browser.visit(url)

In [4]:
addresses = []
properties_info = []

pages = np.arange(0, 30, 1)

for page in pages:
    
#     page_url = "http://house.speakingsame.com/p.php?q=Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    page_url = "http://house.speakingsame.com/p.php?q=East+Perth&p="+ str(page) +"&s=1&st=&type=&count=300&region=East+Perth&lat=0&lng=0&sta=wa&htype=&agent=0&minprice=0&maxprice=0&minbed=0&maxbed=0&minland=0&maxland=0"
    
    browser.visit(page_url)
   
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    addr_spans = soup.find_all('span', class_='addr')

    for span in addr_spans:
        addresses.append(span.a.text)
    addresses

    tables = soup.find_all('table', style="font-size:13px") #tables list (except 0,1 index) contians list of properties in the page

    for i in range(2,12):
        try:
            info = []
            trs = tables[i].find_all('tr')
            for tr in trs:
                info.append(tr.td.text)
        except:
            info.append('')
        properties_info.append(info)
        
  
    
properties_info  

[['Sold $539,000 in Aug 2021',
  'Apartment: ',
  'Land size: 110 sqm',
  'Street view | Nearby'],
 ['Sold $295,000 in Aug 2021',
  'Last Sold $265,000 in Aug 2004',
  'Apartment: 2  1  1 ',
  'Land size: 2,425 sqm | Building size: 99 sqm',
  'Agent: Joel Cooper',
  "Superb Blue Chip - Investors Only!. Superbly located 2 bedroom apartment in the heart of Perth's CBD with so much to offer. Enjoy the stunning returns from this amazing sixth floor apartment that...more",
  'Street view | Nearby'],
 ['Sold $1,100,000 in Jul 2021',
  'Last Sold $280,000 in Jun 1996',
  'House: 4  ',
  'Land size: 454 sqm',
  'Agent: Nadija Begovich',
  'Lot/Plan No: D084602    1000more',
  'Street view | Nearby'],
 ['Sold $429,000 in Jul 2021',
  'Rent $450pw in Sep 2018',
  'Unit: 2  2  1 ',
  'Land size: 131 sqm | Building size: 150 sqm',
  'Agent: Brendan Smith',
  'Street view | Nearby'],
 ['Sold $700,000 in Jul 2021',
  'Rent $800pw in Jul 2013',
  'Apartment: 2  2  2 ',
  'Land size: 6,691 sqm | Build

In [5]:
len(properties_info)

300

In [6]:
len(addresses)

300

In [7]:
Properties = []
for i in range(0, len(addresses)):
    
    Property_dict = {}
    Property_dict['address'] = addresses[i]
    
    for j in range (0, len(properties_info[i])):
        
        try:
                    
            if properties_info[i][j].split(' ')[0] == "Sold": 
                Property_dict['price'] = properties_info[i][j].split(' ')[1]
                Property_dict['sold date'] = properties_info[i][j].split('in ')[1]

            if properties_info[i][j].split(' ')[0] in ["Apartment:", "House:", "Townhouse:"]:
                Property_dict['type'] = properties_info[i][j].split(' ')[0].split(':')[0]
                Property_dict['bedrooms'] = properties_info[i][j].split(' ')[1]
                try:
                    Property_dict['bathrooms'] = properties_info[i][j].split(' ')[3]
                except:
                    Property_dict['bathrooms'] = ''
                try:
                    Property_dict['car space'] = properties_info[i][j].split(' ')[5]
                except:
                    Property_dict['car space'] = ''
                    
            if properties_info[i][j].split(' ')[0] == "Agent:":
                Property_dict['agent'] = properties_info[i][j].split(': ')[1]
                
            if properties_info[i][j].split(' ')[0] == "Rent":
                Property_dict['rent'] = properties_info[i][j].split(' ')[1]
                Property_dict['rent date'] = properties_info[i][j].split('in ')[1]
                
            if properties_info[i][j].split(' ')[1] == "size:":
                if properties_info[i][j].split(' ')[0] == "Land":
                    Property_dict['Land size'] = properties_info[i][j].split(' ')[2]
                    try:
                        if properties_info[i][j].split(' ')[5] == "Building":
                            Property_dict['Building size'] = properties_info[i][j].split(' ')[7]
                    except:
                        continue
                elif properties_info[i][j].split(' ')[0] == "Building":
                    Property_dict['Building size'] = properties_info[i][j].split(' ')[2]
            
            
        except:
            continue

  
    Properties.append(Property_dict)
        
Properties

[{'address': '318/2 Moreau Parade',
  'price': '$539,000',
  'sold date': 'Aug 2021',
  'type': 'Apartment',
  'bedrooms': '',
  'bathrooms': '',
  'car space': '',
  'Land size': '110'},
 {'address': '54/193 Hay Street',
  'price': '$295,000',
  'sold date': 'Aug 2021',
  'type': 'Apartment',
  'bedrooms': '2',
  'bathrooms': '1',
  'car space': '1',
  'Land size': '2,425',
  'Building size': '99',
  'agent': 'Joel Cooper'},
 {'address': '30 Joel Terrace',
  'price': '$1,100,000',
  'sold date': 'Jul 2021',
  'type': 'House',
  'bedrooms': '4',
  'bathrooms': '',
  'car space': '',
  'Land size': '454',
  'agent': 'Nadija Begovich'},
 {'address': '10/38 Fielder Street',
  'price': '$429,000',
  'sold date': 'Jul 2021',
  'rent': '$450pw',
  'rent date': 'Sep 2018',
  'Land size': '131',
  'Building size': '150',
  'agent': 'Brendan Smith'},
 {'address': '36/78 Terrace Road',
  'price': '$700,000',
  'sold date': 'Jul 2021',
  'rent': '$800pw',
  'rent date': 'Jul 2013',
  'type': 'Apa

In [8]:
len(Properties)

300

In [9]:
East_Perth_df = pd.DataFrame(Properties)

In [10]:
East_Perth_df

,address,price,sold date,type,bedrooms,bathrooms,car space,Land size,Building size,agent,rent,rent date
0,318/2 Moreau Parade,"$539,000",Aug 2021,Apartment,,,,110,NaN,NaN,NaN,NaN
1,54/193 Hay Street,"$295,000",Aug 2021,Apartment,2,1,1,"2,425",99,Joel Cooper,NaN,NaN
2,30 Joel Terrace,"$1,100,000",Jul 2021,House,4,,,454,NaN,Nadija Begovich,NaN,NaN
3,10/38 Fielder Street,"$429,000",Jul 2021,NaN,NaN,NaN,NaN,131,150,Brendan Smith,$450pw,Sep 2018
4,36/78 Terrace Road,"$700,000",Jul 2021,Apartment,2,2,2,"6,691",100,Jasmyn Wright,$800pw,Jul 2013
...,...,...,...,...,...,...,...,...,...,...,...,...
295,44/20 Royal Street,"$600,000",May 2020,Apartment,2,2,2,"3,722",97,NaN,$550pw,May 2020
296,10/208 Adelaide Terrace,"$411,000",May 2020,NaN,NaN,NaN,NaN,107,77,NaN,$525pw,May 2017
297,212/106 Terrace Road,"$289,000",May 2020,Apartment,2,1,1,"2,201",52,NaN,$320pw,Mar 2018
298,1/34 East Parade,"$400,000",May 2020,Apartment,1,1,1,93,NaN,NaN,NaN,NaN


In [11]:
East_Perth_df.to_csv("EastPerth_suburb_properties.csv")